In [1]:
import csv
import pandas as pd
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pylab as plt

In [3]:
#real_data = pd.read_csv('./real_data_227.csv')
real_data = pd.read_csv('./all_data.csv')
names = real_data.iloc[:, 0] # name 행

In [4]:
real_csv = open('all_data_forHC.csv', 'w', encoding='utf-8', newline='')
#real_csv = open('real_data_forHC.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(real_csv)
wr.writerow(['name', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'mon_tue', 'tue_wed', 'wed_thu', 'thu_fri', 'fri_sat', 'sat_sun', 'var', 'v_rate', 'days_ends', 'max_rate', 'min_rate', 'count', 'max_ends', 'min_ends', 'total_rate'])

idx = 0
for i in range(0, len(real_data)):
    data = real_data.iloc[i, 1:8] # 걸음 수 데이터
    one_name = real_data.iloc[i, 0] # 이름
    array = []

    # 주중 평균과 주말 평균 간의 증감률
    weekdays = real_data.iloc[i, 1:6]  # 주중
    weekends = real_data.iloc[i, 6:8]  # 주말
    days_means = sum(weekdays, 0.0)/len(weekdays) # 주중 평균
    ends_means = sum(weekends, 0.0)/len(weekends) # 주말 평균
    days_ends = ((ends_means - days_means)/days_means)*100

    mon = real_data.iloc[i, 1]
    tue = real_data.iloc[i, 2]
    wed = real_data.iloc[i, 3]
    thu = real_data.iloc[i, 4]
    fri = real_data.iloc[i, 5]
    sat = real_data.iloc[i, 6]  # 토요일 걸음수
    sun = real_data.iloc[i, 7]  # 일요일 걸음수
    
    # 분산 구하기
    a = real_data.iloc[i, 1:8]*0.001
    v = np.var(a) # 분산
    v_rate = 0
    if v < 0.35:
        v_rate = 100
        
    count = 0
    # 증감률 구하기
    for j in range(2, 8):
        prev = real_data.iloc[i, j-1] # j-1 : 1, 2, 3, 4, 5, 6
        next = real_data.iloc[i, j] # j : 2, 3, 4, 5, 6, 7
        rate = ((next - prev)/prev)*100
        if rate > 0:
            count += 1
        elif rate < 0:
            count -= 1
        array.append(rate)
    
    # 연속적인 증가/감소를 위한 변수
    if count != 6 and count != -6:
        count = 0
    elif count == 6:
        count = 100
    elif count == -6:
        count = -100
    
    ldata = list(data)
    aver = sum(data) / 7
    mini = min(data) # 최대값
    maxi = max(data) # 최소값
    
    # 주말 증가/감소를 위한 변수
    max_ends = 0
    min_ends = 0
    if (maxi == sat or maxi == sun) and days_means < sat and days_means < sun:
        max_ends = 100
    if (mini == sat or mini == sun) and days_means > sat and days_means > sun:
        min_ends = -100
    
    ldata.remove(mini)
    ldata.remove(maxi)
    
    smini = min(ldata)
    smaxi = max(ldata)
    
    max_rate = abs((maxi - smaxi) / aver * 100) # 최대값과 두번째 최대값의 증감률
    min_rate = abs((mini - smini) / aver * 100) # 최소값과 두번째 최소값의 증감률
    
    if v_rate == 100: # 분산이 충분히 작으면
        # 나머지 변수들에 영향을 받지 않도록 0으로
        count = 0
        max_ends = 0
        min_ends = 0
    
    if max_ends == 100 or min_ends == -100: # 주말 증가/감소인 경우
        max_rate = 0
        min_rate = 0
    
    total_rate = 0
    if max_rate >= 50 and min_rate < 25: # 특정 요일 증가
        total_rate = 100
    elif max_rate < 50 and min_rate >= 25: # 특정 요일 감소
        total_rate = -100  
    
    # csv로 저장
    wr.writerow([one_name, mon, tue, wed, thu, fri, sat, sun, array[0], array[1], array[2], array[3], array[4], array[5], v, v_rate, days_ends, max_rate, min_rate, count, max_ends, min_ends, total_rate])

print("finish")
real_csv.close()

finish
